In [1]:
import sys

In [2]:
#sys.setrecursionlimit(22500)

In [3]:
print(sys.getrecursionlimit())

3000


In [4]:
import numpy as np
import jax.numpy as jnp
import jax
from functools import partial

jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")

In [5]:
# @jax.jit
# @partial(jax.jit, static_argnames=['ndim', 'minpts', 'maxpts', 'functn'])
#def jax_cubature(a, b, minpts, maxpts, functn, rel_tol):
#    ndim = len(a)
#@partial(jax.jit, static_argnames=['ndim','functn'])#, 'minpts', 'maxpts', 'functn'])
def jax_cubature(*, ndim : int, a : jnp.ndarray, b : jnp.ndarray, minpts : int, maxpts : int, functn : callable, rel_tol : float) -> tuple:
    
    if ndim < 2:
        raise ValueError("ndim must be greater than 2")
    
    rgnstr  = jnp.int64(2*ndim + 2)
    divaxo  = jnp.int64(0)
    divaxn  = jnp.int64(0)

    #Initialize the cubature
    
    
    rulcls  = jnp.int64(0)
    twondim = jnp.int64(0)
    lambdas = jnp.zeros(3)
    weights = jnp.zeros(5)
    weightsp = jnp.zeros(4)
    ratio = jnp.float64(0.0)



    z      = jnp.zeros(ndim)
    center = jnp.zeros(ndim)
    width  = jnp.zeros(ndim)
    widthl = jnp.zeros(ndim)


    subtmp = jnp.int64(0)
    
    funcls = jnp.int64(0)
    finest = jnp.float64(0.0)
    ifail  = jnp.int64(3)
    

    width  = (b-a)/2.0
    center = a + width

    finest = jnp.float64(0.0)
    subrgn = rgnstr
    sbrgns = rgnstr+1
    divflg = jnp.int64(1)
    relerr = jnp.float64(1.0)
    rgnerr = jnp.float64(0.0)
    rgnval = jnp.float64(0.0)
    rgncmp = jnp.float64(0.0)


    tmpflg = 0
    loopflg = True


    all_vars = {}
    all_vars['ndim'] = ndim
    all_vars['maxpts'] = maxpts
    all_vars['minpts'] = minpts
    
    all_vars['twondim'] = twondim
    all_vars['rgnstr'] = rgnstr
    all_vars['divaxo'] = divaxo
    all_vars['divaxn'] = divaxn

    all_vars['rulcls'] = rulcls
    all_vars['twondim'] = twondim
    all_vars['ratio'] = ratio
    all_vars['lambdas'] = lambdas
    all_vars['weights'] = weights
    all_vars['weightsp'] = weightsp

    all_vars['z'] = z
    all_vars['center'] = center
    all_vars['width']  = width

   

    all_vars['subtmp'] = subtmp
    all_vars['sbtmpp'] = subtmp
    all_vars['funcls'] = funcls
    all_vars['finest'] = finest
    all_vars['ifail']  = ifail
    
    all_vars['width']  = width
    all_vars['center'] = center
    all_vars['rgnvol'] = 2**ndim

    all_vars['subrgn'] = subrgn
    all_vars['sbrgns'] = sbrgns
    all_vars['divflg'] = divflg
    all_vars['relerr'] = relerr
    all_vars['rgnerr'] = rgnerr
    all_vars['rgnval'] = rgnval
    all_vars['rgncmp'] = rgncmp

    all_vars['loopflg'] = loopflg
    all_vars['tmpflg']  = tmpflg
    all_vars['shrink']  = True
    all_vars['flag']    = True

    all_vars['maxit_rule'] = 1 # 100
    all_vars['maxit_ord']  = 1 # 100
    all_vars['max_glob_it'] = 1 #all_vars['maxpts']//all_vars['rulcls']

    all_vars['sum1']   = jnp.float64(0.0)
    all_vars['sum2']   = jnp.float64(0.0)
    all_vars['sum3']   = jnp.float64(0.0)
    all_vars['sum4']   = jnp.float64(0.0)
    all_vars['sum5']   = jnp.float64(0.0)
    all_vars['difmax'] = -1.0
    all_vars['dif']    = 0.0
    all_vars['j']      = 0

    all_vars = initialise(all_vars = all_vars, ndim = ndim)
    lenwrk = jnp.int64((2*ndim+3)*(1+all_vars['maxpts']//all_vars['rulcls'])//2)
    wrkstr = jnp.zeros(lenwrk+1,dtype=jnp.float64)
    all_vars['widthl'] = widthl
    all_vars['lenwrk'] = lenwrk
    all_vars['wrkstr'] = wrkstr

    
    all_vars['tmpflg'] = 0
    while all_vars['ifail']>2:
        if tmpflg!=0:
            all_vars= prepare_new_call(all_vars=all_vars,ndim=ndim)
    
        for i in range(2):    
            if all_vars['tmpflg']==0:
                all_vars['tmpflg'] = 1
            

            #Run Basic Rule
            all_vars = basic_rule(all_vars=all_vars, functn=functn,ndim=ndim)
            #Order and store results of basic rule
            all_vars = order_results(all_vars=all_vars,ndim=ndim)
            #When divflg=0, prepare to call basic rule on second half of subregion
            if all_vars['divflg']!=0:
                all_vars['center'] = all_vars['center'].at[all_vars['divaxo']].set(all_vars['center'][all_vars['divaxo']] + 2.0*all_vars['width'][all_vars['divaxo']])
                all_vars['sbrgns'] = all_vars['sbrgns'] + all_vars['rgnstr'] + 1
                all_vars['subrgn'] = all_vars['sbrgns'] - 1
                all_vars['divflg'] = 1
            else:
                break

            
    all_vars['minpts'] = all_vars['funcls']
    all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['lenwrk']-1].set(all_vars['sbrgns'])
    return all_vars['finest'], all_vars['relerr'], all_vars['minpts'], all_vars['ifail'], all_vars['wrkstr'], all_vars['funcls']
       
        
            
        
        




In [6]:
#@partial(jax.jit, static_argnames=['ndim','rulcls','twondim','lambdas','weights','weightsp'])
#@jax.jit
#@partial(jax.jit, static_argnames=['ndim'])
#def initialise(ndim):
def initialise(*,all_vars : dict,ndim : int)->tuple:
    twondim = 2.0**ndim
    
    lambda5 = 9.0/19.0 
    
    def _if_true(_):
        rulcls = 2**ndim + 2*ndim*ndim + 2*ndim +1
        lambda4 = 9.0/10.0
        lambda2 = 9.0/70.0
        weight5 = 1.0/(3.0*lambda5)**3 /twondim 
        return (rulcls,lambda4,lambda2,weight5)

    def _if_false(_):
        rulcls = 1 + (ndim*(12+(ndim-1)*(6+(ndim-2)*4)))//3
        ratio = (ndim-2)/9.0
        lambda4 = (1.0/5.0 -ratio)/(1.0/3.0 -ratio/lambda5)
        ratio = (1.0 -lambda4/lambda5)*(ndim-1)*ratio/6.0
        lambda2 = (1.0/7.0 -lambda4/5.0 -ratio)/(1.0/5.0 -lambda4/3.0 -ratio/lambda5)

        weight5 = 1.0/(6.0*lambda5)**3
        return (rulcls,lambda4,lambda2,weight5)
    
    params = (ndim,twondim)
    rulcls,lambda4,lambda2,weight5 = jax.lax.cond(ndim<=15, _if_true, _if_false, params)

    weight4 = (1.0/15.0 -lambda5/9.0)/(4.0*(lambda4-lambda5)*lambda4**2)
    weight3 = (1.0/7.0 -(lambda5+lambda2)/5.0 +lambda5*lambda2/3.0)/(2.0*lambda4*(lambda4-lambda5)*(lambda4-lambda2)) -2.0*(ndim-1)*weight4
    weight2 = (1.0/7.0 -(lambda5+lambda4)/5.0 +lambda5*lambda4/3.0)/(2.0*lambda2*(lambda2-lambda5)*(lambda2-lambda4)) 

    tmp = (ndim,weight2,weight3,weight4,weight5,twondim)
    def _if_true(_):
        ndim,weight2,weight3,weight4,weight5,twondim = tmp
        weight1 = 1.0 -2.0*ndim*(weight2+weight3+(ndim-1)*weight4)-twondim*weight5
        return weight1
    def _if_false(_):
        ndim,weight2,weight3,weight4,weight5,twondim = tmp
        weight1 = 1.0 -ndim*(weight2+weight3+(ndim-1)*(weight4+2.0*(ndim-2)*weight5/3.0))
        return weight1
    weight1 = jax.lax.cond(ndim<=15, _if_true, _if_false, tmp)

    weight4p = 1.0/(6.0*lambda4)**2
    weight3p = (1.0/5.0 -lambda2/3.0)/(2.0*lambda4*(lambda4-lambda2)) -2.0*(ndim-1)*weight4p
    weight2p = (1.0/5.0 -lambda4/3.0)/(2.0*lambda2*(lambda2-lambda4))
    weight1p = 1.0 -2.0*ndim*(weight2p+weight3p+(ndim-1)*weight4p)

    ratio = lambda2/lambda4

    lambda5 = jnp.sqrt(lambda5)
    lambda4 = jnp.sqrt(lambda4)
    lambda2 = jnp.sqrt(lambda2)

    lambdas  = jnp.array([lambda2, lambda4, lambda5])
    weights  = jnp.array([weight1, weight2, weight3, weight4, weight5])
    weightsp = jnp.array([weight1p, weight2p, weight3p, weight4p])


    all_vars['rulcls'] = rulcls
    all_vars['twondim'] = twondim
    all_vars['lambdas'] = lambdas
    all_vars['weights'] = weights
    all_vars['weightsp'] = weightsp
    all_vars['ratio'] = ratio
    
    return all_vars


In [7]:
#@jax.jit
#def prepare_new_call(all_vars):
#@partial(jax.jit,static_argnames=['ndim'])
def prepare_new_call(*,all_vars : dict, ndim : int)->dict:
    all_vars['divflg'] = jnp.int64(0)
    all_vars['subrgn'] = all_vars['rgnstr']
    all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['lenwrk']].set(all_vars['wrkstr'][all_vars['lenwrk']] - all_vars['wrkstr'][all_vars['subrgn']])
    all_vars['finest'] = all_vars['finest'] - all_vars['wrkstr'][all_vars['subrgn']-1]
    all_vars['divaxo'] = jnp.int64(all_vars['wrkstr'][all_vars['subrgn']-2])
    #divaxo = jnp.int64(wrkstr[subrgn-2])
    #for j in range(all_vars['ndim']):
    for j in range(ndim):
        all_vars['subtmp'] = all_vars['subrgn']-2*(j+2)
        all_vars['center'] = all_vars['center'].at[j].set(all_vars['wrkstr'][all_vars['subtmp']+1])
        all_vars['width']  = all_vars['width'].at[j].set(all_vars['wrkstr'][all_vars['subtmp']])
        
    all_vars['width']  = all_vars['width'].at[all_vars['divaxo']].set(all_vars['width'][all_vars['divaxo']]/2.0)
    all_vars['center'] = all_vars['center'].at[all_vars['divaxo']].set(all_vars['center'][all_vars['divaxo']]-all_vars['width'][all_vars['divaxo']])

    return all_vars


In [8]:
#@partial(jax.jit, static_argnames=['ndim', 'rulcls', 'twondim', 'functn','maxit'])
#@jax.jit
#@partial(jax.jit, static_argnames=['ndim','functn'])
def basic_rule(*,all_vars : dict, functn : callable, ndim : int)->dict:
#def basic_rule(all_vars,functn,ndim):
    all_vars['rgnvol'] = all_vars['twondim']*jnp.ones_like(all_vars['width'])
    for j in range(ndim):
        all_vars['rgnvol'] = all_vars['rgnvol']*all_vars['width'][j]
        all_vars['z'] = all_vars['z'].at[j].set(all_vars['center'][j])

    all_vars['sum1'] = functn(all_vars['z'])
    #Compute the symetric sums of functn(lambda2,0,0,..0) and functn(lambda4,0,0,..0), and 
    #maximum fourth difference
    all_vars['difmax'] = -1.0
    all_vars['sum2'] = 0.0
    all_vars['sum3'] = 0.0
    for j in range(ndim):
        all_vars['z'] = all_vars['z'].at[j].set(all_vars['center'][j]-all_vars['lambdas'][0]*all_vars['width'][j])
        f1 = functn(all_vars['z'])
        all_vars['z'] = all_vars['z'].at[j].set(all_vars['center'][j]+all_vars['lambdas'][0]*all_vars['width'][j])
        f2 = functn(all_vars['z'])
        all_vars['widthl'] = all_vars['widthl'].at[j].set(all_vars['lambdas'][1]*all_vars['width'][j])
        all_vars['z'] = all_vars['z'].at[j].set(all_vars['center'][j]-all_vars['widthl'][j])
        f3 = functn(all_vars['z'])
        all_vars['z']= all_vars['z'].at[j].set(all_vars['center'][j]+all_vars['widthl'][j])
        f4 = functn(all_vars['z'])
        all_vars['sum2'] = all_vars['sum2'] + f1 + f2
        all_vars['sum3'] = all_vars['sum3'] + f3 + f4
        df1 = f1+f2-2.0*all_vars['sum1']
        df2 = f3+f4-2.0*all_vars['sum1']
        all_vars['dif'] = jnp.fabs(df1-all_vars['ratio']*df2)

        all_vars['j'] = j
        def _if_update(all_vars):
            def _update(_):
                return jnp.int64(all_vars['j'])
            def _no_update(_):
                return jnp.int64(all_vars['divaxn'])
            all_vars['divaxn'] = jax.lax.cond(all_vars['difmax']<all_vars['dif'],_update,_no_update,None)
            return all_vars
        all_vars = _if_update(all_vars)

        def _if_update(all_vars):
            def _update(_):
                return all_vars['dif']
            def _no_update(_):
                return all_vars['difmax']
            all_vars['difmax'] = jax.lax.cond(all_vars['difmax']<all_vars['dif'],_update,_no_update,all_vars)
            return all_vars
        
        all_vars = _if_update(all_vars)
        all_vars['z'] = all_vars['z'].at[j].set(all_vars['center'][j])

    def _if_cond(all_vars):
        def _update(_):
            #return jnp.int64((all_vars['divaxo']+1)%all_vars['ndim'])
            return jnp.int64((all_vars['divaxo']+1)%ndim)
        def _no_update(_):
            return jnp.int64(all_vars['divaxn'])
        all_vars['divaxn'] = jax.lax.cond(all_vars['sum1'] == all_vars['sum1']+all_vars['difmax']/8.0,_update,_no_update,all_vars)
        return all_vars
    all_vars['divaxn'] = _if_cond(all_vars)
    
    #Compute the symetric sums of functn(lambda4,lambda4,0,..0)
    all_vars['sum4'] = 0.0
    for j in range(1,ndim):
        for k in range(j,ndim):
            for l in range(2):
                all_vars['widthl'] = all_vars['widthl'].at[j-1].set(-all_vars['widthl'][j-1])
                all_vars['z'] = all_vars['z'].at[j-1].set(all_vars['center'][j-1]+all_vars['widthl'][j-1])

                for m in range(2):
                    all_vars['widthl'] = all_vars['widthl'].at[k].set(-all_vars['widthl'][k])
                    all_vars['z'] = all_vars['z'].at[k].set(all_vars['center'][k]+all_vars['widthl'][k])
                    f1 = functn(all_vars['z'])
                    all_vars['sum4'] = all_vars['sum4'] + f1
            
            all_vars['z'] = all_vars['z'].at[k].set(all_vars['center'][k])
        all_vars['z'] = all_vars['z'].at[j-1].set(all_vars['center'][j-1])

    #Compute symmetric sum of functn(lambda5,lambda5,lambda5,0,0...0)
    all_vars['sum5'] = 0.0
    
    if ndim<=15:

        all_vars['widthl'] = -all_vars['lambdas'][2]*all_vars['width']
        all_vars['z'] = all_vars['center']+all_vars['widthl']
        
        all_vars['shrink'] = True
        def _loop(all_vars):
            def _outer_if_true(all_vars):
                all_vars['shrink'] = False
                f1 = functn(all_vars['z'])
                all_vars['sum5'] = all_vars['sum5'] + f1
                
                def _body_loop(j,all_vars):
                    all_vars['j'] = j
                    def _if_false(all_vars):

                        def _inner_if_true(all_vars):
                            all_vars['widthl'] = all_vars['widthl'].at[j].set(-all_vars['widthl'][j])
                            all_vars['z'] = all_vars['z'].at[j].set(all_vars['center'][j]+all_vars['widthl'][j])
                            return all_vars

                        def _inner_if_false(all_vars):
                            return all_vars
                        
                        all_vars = jax.lax.cond(all_vars['flag'],_inner_if_true,_inner_if_false,tmp)
                        return all_vars

                    def _if_true(all_vars):
                        def _inner_if_true(all_vars):
                            all_vars['widthl'] = all_vars['widthl'].at[j].set(-all_vars['widthl'][j])
                            all_vars['z'] = all_vars['z'].at[j].set(all_vars['center'][j]+all_vars['widthl'][j])
                            all_vars['flag'] = False
                            all_vars['shrink'] = True
                            return all_vars

                        def _inner_if_false(all_vars):
                            return all_vars
                        
                        all_vars = jax.lax.cond(all_vars['flag'],_inner_if_true,_inner_if_false,all_vars)
                        return all_vars
                    
                    all_vars = jax.lax.cond(all_vars['widthl'][j]<0.0,_if_true,_if_false,all_vars)
                    return all_vars


                all_vars['flag'] = True
                #all_vars = jax.lax.fori_loop(0,all_vars['ndim'],_body_loop,all_vars)
                all_vars = jax.lax.fori_loop(0,ndim,_body_loop,all_vars)

                return all_vars
            
            def _outer_if_false(all_vars):
                return all_vars
            
            all_vars = jax.lax.cond(all_vars['shrink'],_outer_if_true,_outer_if_false,all_vars)
            return all_vars


        all_vars = jax.lax.fori_loop(0,all_vars['maxit_rule'],_loop,all_vars)
        
    else:
        for j in range(ndim):
            all_vars['widthl'] = all_vars['widthl'].at[j].set(all_vars['lambdas'][2]*all_vars['width'][j])
        for i in range(2,ndim):
            for j in range(i,ndim):
                for k in range(j,ndim):
                    for l in range(2):
                        all_vars['widthl'] = all_vars['widthl'].at[i-2].set(-all_vars['widthl'][i-2])
                        all_vars['z'] = all_vars['z'].at[i-2].set(all_vars['center'][i-2]+all_vars['widthl'][i-2])
                        for m in range(2):
                            all_vars['widthl'] = all_vars['widthl'].at[j-1].set(-all_vars['widthl'][j-1])
                            all_vars['z'] = all_vars['z'].at[j-1].set(all_vars['center'][j-1]+all_vars['widthl'][j-1])
                            for n in range(2):
                                all_vars['widthl'] = all_vars['widthl'].at[k].set(-all_vars['widthl'][k])
                                all_vars['z']= all_vars['z'].at[k].set(all_vars['center'][k]+all_vars['widthl'][k])
                                f1 = functn(all_vars['z'])
                                all_vars['sum5'] = all_vars['sum5'] + f1
                    
                        all_vars['z'] = all_vars['z'].at[k].set(all_vars['center'][k])
                    all_vars['z'] = all_vars['z'].at[j-1].set(all_vars['center'][j-1])
                all_vars['z'] = all_vars['z'].at[i-2].set(all_vars['center'][i-2])

    #Compute fifth and seventh degree rules and error.
    all_vars['rgncmp'] = all_vars['rgnvol'] *(all_vars['weightsp'][0]*all_vars['sum1'] + all_vars['weightsp'][1]*all_vars['sum2'] + all_vars['weightsp'][2]*all_vars['sum3'] + all_vars['weightsp'][3]*all_vars['sum4'])
    all_vars['rgnval'] = all_vars['rgnvol'] *(all_vars['weights'][0]*all_vars['sum1'] + all_vars['weights'][1]*all_vars['sum2'] + all_vars['weights'][2]*all_vars['sum3'] + all_vars['weights'][3]*all_vars['sum4'] + all_vars['weights'][4]*all_vars['sum5'])
    all_vars['rgnerr'] = jnp.fabs(all_vars['rgnval']-all_vars['rgncmp'])

    all_vars['finest'] = all_vars['finest']+all_vars['rgnval']
    all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['lenwrk']].set(all_vars['wrkstr'][all_vars['lenwrk']]+all_vars['rgnerr'])
    all_vars['funcls'] = all_vars['funcls']+all_vars['rulcls']

    return all_vars


In [9]:
#@partial(jax.jit, static_argnames=['ndim'])
def order_resuts(*,all_vars : dict, ndim : int)->dict:
#def order_results(all_vars):
    
    def _place_first(all_vars):
    #When divflg=0, start at top of list and move down
    #list tree to find correct position for results from 
    #first half of recently divided subregion
        all_vars['subtmp'] = 2*all_vars['subrgn'] +1
        
        def _body_while(_,all_vars):
        #while (subtmp<sbrgns and rgnerr<wrkstr[subtmp]): 
            def _outer_while_true(_):
                def _while_true(_):
                    def _true(_):
                        all_vars['sbtmpp'] = all_vars['subtmp']+all_vars['rgnstr']+1          
                        def _true_statement(_):
                            return all_vars
                        def _false_statement(_):
                            return all_vars
                        all_vars = jax.lax.cond(all_vars['wrkstr'][all_vars['subtmp']]<all_vars['wrkstr'][all_vars['sbtmpp']],_true_statement,_false_statement,None)               
                        return all_vars
                    
                    def _false(_):
                        return all_vars
        
                    all_vars = jax.lax.cond(all_vars['subtmp']!=all_vars['sbrgns']-1,_true,_false,all_vars)

                    def loop_body(k,all_vars):
                        all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['subrgn']-k].set(all_vars['wrkstr'][all_vars['subtmp']-k])
                        return all_vars
                    all_vars = jax.lax.fori_loop(0,all_vars['rgnstr']+1,loop_body,all_vars)        

                    all_vars['subrgn'] = all_vars['subtmp']
                    all_vars['subtmp'] = 2*all_vars['subrgn'] +1
                    return all_vars
                
                
                def _while_false(_):
                    return all_vars

                cond2 = all_vars['subtmp']<all_vars['sbrgns'] 
                all_vars = jax.lax.cond(cond2,_while_true,_while_false,all_vars)
                return all_vars
            

            def _outer_while_false(_):
                    return all_vars


            cond1 = all_vars['rgnerr']<all_vars['wrkstr'][all_vars['subtmp']]
            all_vars = jax.lax.cond(cond1,_outer_while_true,_outer_while_false,all_vars)
            return all_vars

        all_vars = jax.lax.fori_loop(0,all_vars['maxit_ord'],_body_while,all_vars)
        return all_vars


    def _place_second(all_vars):
    #When divflg=1, start at bottom right branch and move
    #up list tree to find correct position for results from
    #second half of recently divided subregion
        all_vars['subtmp'] = ((all_vars['subrgn']+1)//(2*(all_vars['rgnstr']+1)))*(all_vars['rgnstr']+1)-1
        cond1 = all_vars['subtmp']>=all_vars['rgnstr']
        cond2 = all_vars['rgnerr'] > all_vars['wrkstr'][all_vars['subtmp']]
        
        def _body_while(_,all_vars):
            def _outer_while_true(_):
                def _while_true(_):
                    def _loop_body(k,all_vars):
                        all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['subrgn']-k].set(all_vars['wrkstr'][all_vars['subtmp']-k])
                        return all_vars
                    all_vars = jax.lax.fori_loop(0,all_vars['rgnstr']+1,_loop_body,all_vars)


                    all_vars['subrgn'] = all_vars['subtmp']
                    all_vars['subtmp'] = ((all_vars['subrgn']+1)//(2*(all_vars['rgnstr']+1)))*(all_vars['rgnstr']+1)-1
                    return all_vars
                def _while_false(_):
                    return all_vars
                cond1 = all_vars['subtmp']>=all_vars['rgnstr']
                all_vars = jax.lax.cond(cond1,_while_true,_while_false,None)
                return all_vars
        
            def _outer_while_false(_):
                return all_vars
            
            cond2 = all_vars['rgnerr'] > all_vars['wrkstr'][all_vars['subtmp']]
            all_vars = jax.lax.cond(cond2,_outer_while_true,_outer_while_false,None)
            return all_vars


        all_vars = jax.lax.fori_loop(0,all_vars['maxit_ord'],_body_while,all_vars)
        return all_vars

 

    #if  divflg !=1:
    #    wrkstr,subrgn,subtmp = _place_first(subrgn,sbrgns,rgnstr,rgnerr,wrkstr)    

    #else: 
    #    wrkstr,subrgn,subtmp = _place_second(subrgn,rgnstr,rgnerr,wrkstr)
    cond = all_vars['divflg']!=1
    all_vars = jax.lax.cond(cond,_place_first,_place_second,all_vars)

    #Store results of basic rule in correct position in list
    all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['subrgn']].set(all_vars['rgnerr'])
    all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['subrgn']-1].set(all_vars['rgnval'])
    all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['subrgn']-2].set(all_vars['divaxn'])
    #for j in range(all_vars['ndim']):
    for j in range(ndim):
        all_vars['subtmp'] = all_vars['subrgn']-2*(j+2)
        all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['subtmp']+1].set(all_vars['center'][j])
        all_vars['wrkstr'] = all_vars['wrkstr'].at[all_vars['subtmp']].set(all_vars['width'][j])

    return all_vars

In [10]:
#@partial(jax.jit, static_argnames=['ndim'])
#@jax.jit
def check_convergence(*,all_vars : dict) -> dict:
#def check_convergence(all_vars):
    def _if_true(_):
        all_vars['wkrstr'] = all_vars['wrkstr'].at[all_vars['lenwrk']].set(0.0)
        return all_vars
    def _if_false(_):
        return all_vars
    all_vars = jax.lax.cond(all_vars['wrkstr'][all_vars['lenwrk']] <= 0.0,_if_true,_if_false,all_vars)

    #if jnp.fabs(finest) != 0.0:
    def _if_true(_):
        all_vars['relerr'] = all_vars['wrkstr'][all_vars['lenwrk']]/jnp.fabs(all_vars['finest'])
        return all_vars
    def _if_false(_):
        return all_vars
    all_vars = jax.lax.cond(jnp.fabs(all_vars['finest']) != 0.0,_if_true,_if_false,all_vars)


    #if 1.0 < relerr:
    def _if_true(_):
        all_vars['relerr'] = 1.0
        return all_vars
    def _if_false(_):
        return all_vars
    all_vars = jax.lax.cond(1.0 < all_vars['relerr'],_if_true,_if_false,all_vars)

    #if lenwrk<(sbrgns+rgnstr+2):
    def _if_true(_):
        all_vars['ifail']=2
        return all_vars
    def _if_false(_):
        return all_vars
    all_vars = jax.lax.cond(all_vars['lenwrk']<(all_vars['sbrgns']+all_vars['rgnstr']+2),_if_true,_if_false,all_vars)
    
    #if maxpts<(funcls+2*rulcls):
    def _if_true(_):
        all_vars['ifail']=1
        return all_vars
    def _if_false(_):
        return all_vars
    all_vars = jax.lax.cond(all_vars['maxpts']<(all_vars['funcls']+2*all_vars['rulcls']),_if_true,_if_false,all_vars)
 
    #if (relerr<rel_tol) and (minpts<=funcls):
    cond1 = all_vars['relerr']<all_vars['rel_tol']
    cond2 = all_vars['minpts']<=all_vars['funcls']
    def _outer_if_true(_):
        def _if_true(_):
            all_vars['ifail']=0
            return all_vars
        def _if_false(_):
            return all_vars
        all_vars = jax.lax.cond(cond2,_if_true,_if_false,all_vars)
        return all_vars
    def _outer_if_false(_):
        return all_vars
    all_vars = jax.lax.cond(cond1,_outer_if_true,_outer_if_false,all_vars)

    return all_vars

In [11]:
from cubature import cubature
from numba import jit

@jit(nopython=True)
def test_function(x_array):
    x = x_array[0]
    y = x_array[1]
    z = x_array[2]
    return x**2 +np.log10(y+2)**2.5 + x*z**np.log(2)
#@jax.jit
def jax_test_function(x_array):
    x = x_array[0]
    y = x_array[1]
    z = x_array[2]
    return x**2 +jnp.log10(y+2)**2.5 + x*z**jnp.log(2)

# Integration limits
a = np.array([0, 0, 0])
b = np.array([np.pi, np.pi, 1])

a_jax = jnp.array([0, 0, 0])
b_jax = jnp.array([jnp.pi, jnp.pi, 1])

In [12]:
import numpy as npy
result_cb, error_cb = cubature(test_function, ndim=3, fdim=1, xmin=np.array(a), xmax=np.array(b),
                               relerr=1e-12)
print(f'Result with cubature: {result_cb}, Estimated error: {error_cb}')

Result with cubature: [43.89958661], Estimated error: [9.83865983e-09]


In [12]:
# Uso de numpy_cubature
minpts = 10
maxpts = 500
rel_tol = 1e-8
ndim = 2
#finest, relerr, minpts, ifail, wrkstr, funcls = numpy_cubature(a,b, minpts, maxpts, test_function, rel_tol)
finest, relerr, minpts, ifail, wrkstr, funcls = jax_cubature(ndim=ndim,a=a_jax,b=b_jax, minpts=minpts, maxpts=maxpts, functn=jax_test_function, rel_tol=rel_tol)

print(f'Result with numpy_cubature: {finest}, Estimated error: {relerr}, ifail : {ifail}')
print(funcls)

Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


In [ ]:
all_vars = {'a':3}
initialise(ndim=3,all_vars = all_vars)

In [ ]:
jnp.zeros(all_vars['rulcls'])

In [ ]:
ndim = 3
maxpts = 10000
all_vars = initialise(ndim=ndim,all_vars = {})
lenwrk = jnp.int64((2*ndim+3)*(1+maxpts//all_vars['rulcls'])//2)


In [ ]:
wrkstr = jnp.zeros(lenwrk+1,dtype=jnp.float64)